In [1]:
!pip install wrds
import wrds
import pandas as pd
from scipy.stats.mstats import winsorize

In [2]:
db = wrds.Connection(wrds_username= 'clkride')

Loading library list...
Done


# Step 1: Download All Datasets

#### Import Compustat Dataset from WRDS

In [3]:
# Download the compustat dataset
library = 'comp'
file = 'funda'

# set the required filters
indfmt = 'INDL'
datafmt = 'STD'
popsrc = 'D'
consol = 'C'

# fyear = 2009 to 2023
# set up query
query = f"SELECT * FROM {library}.{file} WHERE indfmt='{indfmt}' AND datafmt='{datafmt}' AND popsrc='{popsrc}' AND consol='{consol}' AND datadate >= '01/01/2009' AND datadate <='12/31/2023'"
# execute query
comp_data = db.raw_sql(query) 

To extract Indusrty Code, merge with company table and get the sic code, then extract first two digits of sic code

In [4]:
# Read in company data
company_data = db.get_table(library='comp', table='company', columns=['gvkey', 'sic'])

# Merge the two dataframes
merged_data = comp_data.merge(company_data, on='gvkey', how='left')

# convert the 'sic' column from strings to integers
merged_data['sic'] = merged_data['sic'].astype(int)

In [5]:
# extract the first two digits of sic column and store them in a new column
merged_data['industry'] = merged_data['sic'].astype(str).str[:2].astype(int)

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\249207560.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data['industry'] = merged_data['sic'].astype(str).str[:2].astype(int)


For lean operation, perform garbage collection to free up memory

In [6]:
import gc
del company_data
# Perform garbage collection
gc.collect()

14

In [7]:
# filter all required columns
df = merged_data[['gvkey','tic','conm','invch','sic','prcc_f','fyear','cusip', 'industry','ib', 'spi', 'at', 'dvc', 'act', 'che', 'lct', 'dlc', 'txp', 'dp', 'csho', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']]

In [8]:
# replace missing values with zeros for the specified columns
df[['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']] = df[['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']].fillna(0)

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\4236409823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']] = df[['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']].fillna(0)


#### Import IBES Dataset

In [9]:
library = 'ibes'
file = 'statsum_epsus'

# set the required filters
FPI = 1
Measure = 'EPS'

# set up query
query_ibes = f"SELECT * FROM {library}.{file} WHERE FPI='{FPI}' AND Measure='{Measure}' AND fpedats BETWEEN '2010-01-01' AND '2022-12-31'"
# execute query
ibes_data = db.raw_sql(query_ibes) 

In [10]:
ibes_data.head()

,ticker,cusip,oftic,cname,statpers,measure,fiscalp,fpi,estflag,curcode,...,highest,lowest,usfirm,fpedats,actual,actdats_act,acttims_act,anndats_act,anntims_act,curr_act
0,0000,87482X10,TLMR,TALMER BANCORP,2014-04-17,EPS,ANN,1,P,USD,...,0.56,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD
1,0000,87482X10,TLMR,TALMER BANCORP,2014-05-15,EPS,ANN,1,P,USD,...,0.58,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD
2,0000,87482X10,TLMR,TALMER BANCORP,2014-06-19,EPS,ANN,1,P,USD,...,0.59,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD
3,0000,87482X10,TLMR,TALMER BANCORP,2014-07-17,EPS,ANN,1,P,USD,...,0.59,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD
4,0000,87482X10,TLMR,TALMER BANCORP,2014-08-14,EPS,ANN,1,P,USD,...,1.24,1.09,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD


#### Import comp_na_daily for cshom and prccm

In [ ]:
# Read in company data
#monthly_data = db.raw_sql(""" select prccm, tic, cshom, datadate from comp_na_daily_all.secm where cyear >= 2010""")

#monthly_data

Extract only rows from month of June

In [ ]:
#monthly_data['datadate'] = pd.to_datetime(monthly_data['datadate'])
#monthly_data = monthly_data[(monthly_data['datadate'].dt.month == 6)]
#monthly_data['fyear'] = monthly_data['datadate'].dt.year

In [ ]:
# drop the 'city' column
#monthly_data = monthly_data.drop('datadate', axis=1)
#monthly_data

In [ ]:
# merge compustat dataset with monthly data to include cshom values
#monthly_data = df.merge(monthly_data, on=['tic','fyear'], how='left')
#monthly_data

In [11]:
# create a boolean mask to filter rows where fyear is less than 2010
mask = df['fyear'] > 2009

# use the boolean mask to filter the dataframe and drop the rows where fyear is less than 2010
df = df[mask].dropna()
df

,gvkey,tic,conm,invch,sic,prcc_f,fyear,cusip,industry,ib,...,dlc,txp,dp,csho,ceq,ivao,lt,dltt,ivst,pstk
2,001004,AIR,AAR CORP,-34.615,5080,26.3900,2010.0,000361105,50,73.139,...,114.075,0.000,59.296,39.781,835.845,2.443,868.438,329.802,0.000,0.000
3,001004,AIR,AAR CORP,-42.057,5080,12.0500,2011.0,000361105,50,67.723,...,122.865,0.000,80.333,40.273,864.649,18.869,1329.631,669.489,0.000,0.000
4,001004,AIR,AAR CORP,26.500,5080,20.0600,2012.0,000361105,50,55.000,...,86.400,0.000,108.600,39.382,918.600,16.800,1217.400,622.200,0.000,0.000
5,001004,AIR,AAR CORP,-36.600,5080,24.3000,2013.0,000361105,50,72.900,...,69.700,0.000,113.400,39.560,999.500,5.200,1198.800,564.300,0.000,0.000
6,001004,AIR,AAR CORP,-36.600,5080,29.5400,2014.0,000361105,50,-54.500,...,69.000,0.000,92.300,35.423,845.100,4.300,669.900,85.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160393,351038,QNRX,CELLECT BIOTECHNOLOGY LTD,0.000,2834,1.4200,2022.0,74907L201,28,-9.381,...,0.000,0.000,0.104,4.847,7.407,0.000,7.051,0.000,9.993,0.000
160397,351491,IVCGF,IVECO GROUP N V,-234.055,3711,5.8800,2022.0,N47017103,37,157.105,...,3786.553,114.355,828.275,271.215,2514.750,167.792,14558.406,951.181,0.000,1.069
160400,351590,DTRUY,DAIMLER TRUCK HOLDING AG,-1486.484,3713,18.4389,2021.0,23384L101,37,2669.303,...,6231.408,276.370,2035.813,1645.904,18106.225,10910.365,43647.149,12647.062,119.419,0.000
160401,351590,DTRUY,DAIMLER TRUCK HOLDING AG,-1221.573,3713,15.4460,2022.0,23384L101,37,2848.198,...,8027.323,184.892,1963.278,1645.904,21430.419,13970.599,46343.870,14244.198,1201.266,0.000


#### Function to calculate all the X variables and Y variable values for HVZ, EP and RI model

In [12]:
# define a function to calculate delta values within each group
def calc_deltas(group):
   # Calculate changes in variables
     group['del_STDebt'] = group['dlc'] - group['dlc'].shift(1) # sign is positive
     group['del_CA'] = group['act'] - group['act'].shift(1) # sign is positive
     group['del_CL'] = group['lct'] - group['lct'].shift(1) # sign is negative
     group['del_Cash'] = group['che'] - group['che'].shift(1) # sign negative
     group['del_txp'] = group['txp'] - group['txp'].shift(1) # sign positive

   # Calculate total current accruals using the equation
     group['ACT'] = group['del_CA'] - group['del_Cash'] - group['del_CL']+ group['del_STDebt'] + group['del_txp'] - group['dp']
     
  # y-variable for HVZ
     group['E_t'] = group['ib']-group['spi']
     group['E_t1_HVZ']= group['ib'].shift(-1)-group['spi'].shift(-1)
     group['E_t2_HVZ']= group['ib'].shift(-2)-group['spi'].shift(-2)
     group['E_t3_HVZ']= group['ib'].shift(-3)-group['spi'].shift(-3)

  # Variables for EP and Model
     # add small number to avoid division by zero
     group['EPS'] = group['E_t']/(group['csho']+ 1e-6)
     group['E_t1_EP_RI']= (group['ib'].shift(-1)-group['spi'].shift(-1))/(group['csho']+ 1e-6)
     group['E_t2_EP_RI']= (group['ib'].shift(-2)-group['spi'].shift(-2))/(group['csho']+ 1e-6)
     group['E_t3_EP_RI']= (group['ib'].shift(-3)-group['spi'].shift(-3))/(group['csho']+ 1e-6)

  # Variables for RI Model
     group['B_t'] = group['ceq']/group['csho']  # Book value 
     group['WC'] = group['act']-group['che']-group['lct']+group['dlc']
     group['NCO']= group['at']-group['act']-group['lt']+group['lct']+group['dltt']
     group['FIN']= group['ivst']-group['dltt']-group['dlc']-group['pstk']
     group['TACC_t'] = (group['WC']+group['NCO']+group['FIN'])/group['csho']

    # Create dummy variables for dividend and retained earnings
     group['dummy_Neg_E_t'] = group['E_t'].apply(lambda x: 1 if x < 0 else 0)
     group['dummy_Div'] = group['dvc'].apply(lambda x: 0 if pd.isnull(x) else 1)
     group['Neg_E_interaction_term'] = group['dummy_Neg_E_t']*group['EPS']
     return group

In [13]:
# group the data by gvkey code
# This is to make sure that the data we use is for one company only and that it doesnot take values from a row above that belongs to some other company
df = df.groupby('gvkey')

In [14]:
# Fill all remaining None Type values with zero
df.fillna(0, inplace=True)

""


In [15]:
# apply the calc_deltas function to each group
delta_df = df.apply(calc_deltas)

In [16]:
delta_df

,gvkey,tic,conm,invch,sic,prcc_f,fyear,cusip,industry,ib,...,E_t2_EP_RI,E_t3_EP_RI,B_t,WC,NCO,FIN,TACC_t,dummy_Neg_E_t,dummy_Div,Neg_E_interaction_term
2,001004,AIR,AAR CORP,-34.615,5080,26.3900,2010.0,000361105,50,73.139,...,1.912973,1.832533,21.011161,554.617,667.116,-443.877,19.553455,0,1,0.000000
3,001004,AIR,AAR CORP,-42.057,5080,12.0500,2011.0,000361105,50,67.723,...,1.810146,-0.151466,21.469694,645.191,945.465,-792.354,19.822263,0,1,0.000000
4,001004,AIR,AAR CORP,26.500,5080,20.0600,2012.0,000361105,50,55.000,...,-0.154893,1.038546,23.325377,655.800,897.000,-708.600,21.436189,0,1,0.000000
5,001004,AIR,AAR CORP,-36.600,5080,24.3000,2013.0,000361105,50,72.900,...,1.033873,1.203236,25.265420,695.300,850.200,-634.000,23.040950,0,1,0.000000
6,001004,AIR,AAR CORP,-36.600,5080,29.5400,2014.0,000361105,50,-54.500,...,1.343760,2.080569,23.857381,556.400,388.000,-154.000,22.313186,1,1,-0.172204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160393,351038,QNRX,CELLECT BIOTECHNOLOGY LTD,0.000,2834,1.4200,2022.0,74907L201,28,-9.381,...,NaN,NaN,1.528162,-3.012,-2.435,9.993,0.937900,1,1,-2.021250
160397,351491,IVCGF,IVECO GROUP N V,-234.055,3711,5.8800,2022.0,N47017103,37,157.105,...,NaN,NaN,9.272164,12454.053,-7606.238,-4738.803,0.401939,0,1,0.000000
160400,351590,DTRUY,DAIMLER TRUCK HOLDING AG,-1486.484,3713,18.4389,2021.0,23384L101,37,2669.303,...,NaN,NaN,11.000778,9934.541,19264.022,-18759.051,6.342722,0,1,0.000000
160401,351590,DTRUY,DAIMLER TRUCK HOLDING AG,-1221.573,3713,15.4460,2022.0,23384L101,37,2848.198,...,NaN,NaN,13.020455,12650.702,24089.451,-21070.255,9.520542,0,1,0.000000


#### winsorize columns at yearly level

In [17]:
# Create an empty data frame to hold the winsorized data
df_win = pd.DataFrame()

# List of columns to winsorize
cols_to_winsorize = ['at','dvc','E_t','dummy_Neg_E_t','ACT','Neg_E_interaction_term','B_t','TACC_t',
                     'E_t1_HVZ','E_t2_HVZ','E_t3_HVZ','E_t1_EP_RI','E_t2_EP_RI','E_t3_EP_RI']

# Winsorize by year
for year in delta_df['fyear'].unique():
    # Subset the data for the current year
    df_year = delta_df[delta_df['fyear'] == year]
    
    # Winsorize the columns for the current year's data
    for col in cols_to_winsorize:
        df_year[col] = winsorize(df_year[col], limits=(0.01, 0.01))
    
    # Append the winsorized data for the current year to df_winsorized
    df_win = pd.concat([df_win, df_year])

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\1103843939.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = winsorize(df_year[col], limits=(0.01, 0.01))
C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\1103843939.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = winsorize(df_year[col], limits=(0.01, 0.01))
C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\1103843939.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

#### Additional Cleaning Steps

In [18]:
# count the number of rows that contain at least one NaN value
nan_count = df_win[cols_to_winsorize].isna().sum(axis=1).astype(bool).sum()

# print the count of rows containing at least one NaN value
print("Number of rows containing at least one NaN value:", nan_count)

Number of rows containing at least one NaN value: 36679


In [19]:
print(df_win['E_t1_HVZ'].isna().sum(), df_win['E_t1_EP_RI'].isna().sum())

11046 11046


We will drop these NAN values during training

In [20]:
import numpy as np
# check for inf values and replace them with NAN for now 
cols_inf = ['EPS', 'E_t1_EP_RI','E_t2_EP_RI','E_t3_EP_RI']
df_win[cols_inf] = df_win[cols_inf].replace([np.inf, -np.inf], np.nan)

#### Split in training and test set

In [119]:
# Training dataset = 2010-2019 Test dataset = 2020, 2021, and 2022
# split data into training and test datasets based on date
train_data = df_win[(df_win['fyear'] >= 2010) & (df_win['fyear'] <= 2018)]
test_data_2019 = df_win[df_win['fyear'] == 2019]
test_data_2020 = df_win[df_win['fyear'] == 2020]
test_data_2021 = df_win[df_win['fyear'] == 2021]
test_data_2022 = df_win[df_win['fyear'] == 2022]

In [120]:
print(train_data.shape[0])
print(test_data_2020.shape[0])
print(test_data_2021.shape[0])
print(test_data_2022.shape[0])

52528
5286
5635
4404


#### Regression Model

In [23]:
import statsmodels.api as sm
# Define Regression Function
def ols(input_data, input_y, input_X):
    Y = input_data[input_y]
    X = input_data[input_X]
    X['intercept'] = 1.0
    model = sm.OLS(Y, X).fit()
    return model

### HVZ model

#### HVZ model for earnings in year E_t+1

In [121]:
hvz_train_t1 = train_data.dropna(subset=['at','dvc','E_t','dummy_Neg_E_t','ACT','E_t1_HVZ'])

# Estimate coefficients for HVZ Model at industry and fyear level
hvz_model_t1 = ols(hvz_train_t1,'E_t1_HVZ',['at','dvc','E_t','dummy_Neg_E_t','ACT'])
params_hvz_model_t1 = hvz_model_t1.params
hvz_model_t1.summary()

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2881450415.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1.0


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               E_t1_HVZ   R-squared:                       0.603
Model:                            OLS   Adj. R-squared:                  0.603
Method:                 Least Squares   F-statistic:                 1.201e+04
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        18:35:28   Log-Likelihood:            -3.2762e+05
No. Observations:               39470   AIC:                         6.553e+05
Df Residuals:                   39464   BIC:                         6.553e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
at                0.0098      0.001     10.860      0.000       0.008       0.012
dvc               0.4608      0.030     15.554      0.000       0.403       0.519
E_t               1.1351      0.014     81.473      0.000       1.108       1.162
dummy_Neg_E_t   128.5473     10.299     12.482      0.000     108.362     148.733
ACT              -0.1564      0.014    -11.536      0.000      -0.183      -0.130
intercept      -126.0783      7.385    -17.073      0.000    -140.552    -111.604
==============================================================================
Omnibus:                    87983.314   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        982131707.831
Skew:                          20.638   Prob(JB):                         0.00
Kurtosis:                     774.679   Cond. No.                     3.47e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.47e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [122]:
# Testing Accuracy on out of sample data
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp
# Select the same columns as used in training the model
hvz_test_t1 = test_data_2019[['gvkey','at','dvc','E_t','dummy_Neg_E_t','ACT']].dropna()
hvz_test_t1 = hvz_test_t1.set_index('gvkey')
# Make predictions using the model for 2020 test data
y_pred_2020 = hvz_test_t1.apply(lambda row: params_hvz_model_t1['intercept'] + \
    params_hvz_model_t1['at']*row['at'] + params_hvz_model_t1['dvc']*row['dvc'] + \
    params_hvz_model_t1['E_t']*row['E_t'] + params_hvz_model_t1['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_t1['ACT']*row['ACT'], axis=1)

# Reset the index of the y_pred_2020 DataFrame
y_pred_2020 = y_pred_2020.reset_index()
y_pred_2020
y_true_2020 = test_data_2020[['gvkey', 'E_t']]
merged_df = y_true_2020.merge(y_pred_2020, on='gvkey')
# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2020
mae_2020 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2020:", mae_2020)
print("MSE for 2020:", mse_2020)
print("RMSE for 2020:", rmse_2020)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()
from scipy.stats import ttest_rel
# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2020: 288.4273379069446
MSE for 2020: 1000252.2045708699
RMSE for 2020: 1000.1260943355443
Mean bias: 180.26547279428527
Median bias: 1.173710421585402
p-value for mean bias: 1.057375202057314e-34
p-value for median bias: 2.7903872681919336e-34
Mean accuracy: 0.6492380679881898
Median accuracy: 1.124919264206976
p-value for mean accuracy: 0.9624994442575054
p-value for median accuracy: 0.9491598315821204
t-statistic: 12.390322200184038
p-value for t-statistic: 1.057375202057314e-34


#### HVZ model for earnings in year E_t+2

In [123]:
hvz_train_t2 = train_data.dropna(subset=['at','dvc','E_t','dummy_Neg_E_t','ACT','E_t2_HVZ'])

# Estimate coefficients for HVZ Model at industry and fyear level
hvz_model_t2 = ols(hvz_train_t2,'E_t2_HVZ',['at','dvc','E_t','dummy_Neg_E_t','ACT'])
params_hvz_model_t2 = hvz_model_t2.params
hvz_model_t2.summary()

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2881450415.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1.0


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               E_t2_HVZ   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     8858.
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        18:36:25   Log-Likelihood:            -3.0456e+05
No. Observations:               36221   AIC:                         6.091e+05
Df Residuals:                   36215   BIC:                         6.092e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
at                0.0153      0.001     15.034      0.000       0.013       0.017
dvc               0.5520      0.034     16.403      0.000       0.486       0.618
E_t               0.9537      0.016     60.436      0.000       0.923       0.985
dummy_Neg_E_t   112.4974     11.998      9.376      0.000      88.981     136.014
ACT              -0.1564      0.015    -10.200      0.000      -0.186      -0.126
intercept      -110.8751      8.503    -13.040      0.000    -127.541     -94.209
==============================================================================
Omnibus:                    75758.410   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        639596394.188
Skew:                          17.737   Prob(JB):                         0.00
Kurtosis:                     653.029   Cond. No.                     3.57e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.57e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [124]:
# Testing Accuracy on out of sample data
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp
# Select the same columns as used in training the model
hvz_test_t2 = test_data_2019[['gvkey','at','dvc','E_t','dummy_Neg_E_t','ACT']].dropna()
hvz_test_t2 = hvz_test_t2.set_index('gvkey')
# Make predictions using the model for 2020 test data
y_pred_2021 = hvz_test_t2.apply(lambda row: params_hvz_model_t2['intercept'] + \
    params_hvz_model_t2['at']*row['at'] + params_hvz_model_t2['dvc']*row['dvc'] + \
    params_hvz_model_t2['E_t']*row['E_t'] + params_hvz_model_t2['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_t2['ACT']*row['ACT'], axis=1)

# Reset the index of the y_pred_2021 DataFrame
y_pred_2021 = y_pred_2021.reset_index()
y_pred_2021
y_true_2021 = test_data_2021[['gvkey', 'E_t']]
merged_df = y_true_2021.merge(y_pred_2021, on='gvkey')
# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2021
mae_2021 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2021:", mae_2021)
print("MSE for 2021:", mse_2021)
print("RMSE for 2021:", rmse_2021)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()
from scipy.stats import ttest_rel
# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2021: 232.6919477161709
MSE for 2021: 449754.1448010612
RMSE for 2021: 670.6371185679042
Mean bias: 32.59971329773349
Median bias: -0.37595881755670035
p-value for mean bias: 0.0014521566592394718
p-value for median bias: 0.0012785343618017865
Mean accuracy: 0.6053714349167623
Median accuracy: 0.9299385196859302
p-value for mean accuracy: 0.963450318041863
p-value for median accuracy: 0.9699358629460615
t-statistic: 3.18614832376883
p-value for t-statistic: 0.0014521566592394718


#### HVZ model for earnings in year E_t+3

In [125]:
hvz_train_t3 = train_data.dropna(subset=['at','dvc','E_t','dummy_Neg_E_t','ACT','E_t3_HVZ'])

# Estimate coefficients for HVZ Model at industry and fyear level
hvz_model_t3 = ols(hvz_train_t3,'E_t3_HVZ',['at','dvc','E_t','dummy_Neg_E_t','ACT'])
params_hvz_model_t3 = hvz_model_t3.params
hvz_model_t3.summary()

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2881450415.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1.0


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               E_t3_HVZ   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     7256.
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        18:43:43   Log-Likelihood:            -2.8549e+05
No. Observations:               33314   AIC:                         5.710e+05
Df Residuals:                   33308   BIC:                         5.710e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
at                0.0197      0.001     16.184      0.000       0.017       0.022
dvc               0.4749      0.040     11.770      0.000       0.396       0.554
E_t               1.0428      0.019     55.285      0.000       1.006       1.080
dummy_Neg_E_t   135.1292     14.734      9.171      0.000     106.249     164.009
ACT              -0.1365      0.018     -7.505      0.000      -0.172      -0.101
intercept      -125.9376     10.335    -12.186      0.000    -146.194    -105.681
==============================================================================
Omnibus:                    76061.763   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1335437784.363
Skew:                          21.669   Prob(JB):                         0.00
Kurtosis:                     982.895   Cond. No.                     3.68e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.68e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [126]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
hvz_test_t3 = test_data_2019[['gvkey','at','dvc','E_t','dummy_Neg_E_t','ACT']].dropna()
hvz_test_t3 = hvz_test_t3.set_index('gvkey')

# Make predictions using the model for 2022 test data
y_pred_2022 = hvz_test_t3.apply(lambda row: params_hvz_model_t3['intercept'] + \
    params_hvz_model_t3['at']*row['at'] + params_hvz_model_t3['dvc']*row['dvc'] + \
    params_hvz_model_t3['E_t']*row['E_t'] + params_hvz_model_t3['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_t3['ACT']*row['ACT'], axis=1)

# Reset the index of the y_pred_2022 DataFrame
y_pred_2022 = y_pred_2022.reset_index()

y_true_2022 = test_data_2022[['gvkey', 'E_t']]
merged_df = y_true_2022.merge(y_pred_2022, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2022
mae_2022 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2022:", mae_2022)
print("MSE for 2022:", mse_2022)
print("RMSE for 2022:", rmse_2022)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")


MAE for 2022: 299.7624810666547
MSE for 2022: 697544.0074306902
RMSE for 2022: 835.1910005685468
Mean bias: 22.82701373605013
Median bias: 2.1742342969800994
p-value for mean bias: 0.12256550289222676
p-value for median bias: 0.16239275333401818
Mean accuracy: -46.45491153325111
Median accuracy: 0.8293347336836914
p-value for mean accuracy: 0.2574412766845269
p-value for median accuracy: 0.25915624768066126
t-statistic: 1.5445037534573776
p-value for t-statistic: 0.12256550289222676


## Earnings Persistence Model

#### EP model for earnings in year E_t+1

In [127]:
ep_train_t1 = train_data[['dummy_Neg_E_t','Neg_E_interaction_term','EPS','E_t1_EP_RI']].dropna()

# Estimate coefficients for HVZ Model at industry and fyear level
ep_model_t1 = ols(ep_train_t1,'E_t1_EP_RI',['dummy_Neg_E_t','Neg_E_interaction_term','EPS'])
params_ep_model_t1 = ep_model_t1.params
ep_model_t1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t1_EP_RI   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     870.1
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        18:49:41   Log-Likelihood:            -3.3664e+05
No. Observations:               48142   AIC:                         6.733e+05
Df Residuals:                   48138   BIC:                         6.733e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
dummy_Neg_E_t             -5.4602      2.587     -2.110      0.035     -10.531      -0.389
Neg_E_interaction_term    -1.1580      1.886     -0.614      0.539      -4.854       2.538
EPS                       -0.0626      0.001    -51.030      0.000      -0.065      -0.060
intercept                  4.9093      1.672      2.937      0.003       1.633       8.185
==============================================================================
Omnibus:                   227502.160   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    2420571865593.842
Skew:                         177.350   Prob(JB):                         0.00
Kurtosis:                   34739.008   Cond. No.                     2.40e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [128]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
ep_test_t1 = test_data_2019[['gvkey','dummy_Neg_E_t','Neg_E_interaction_term','EPS','E_t1_EP_RI']].dropna()
ep_test_t1 = ep_test_t1.set_index('gvkey')

# Make predictions using the model for 2022 test data
y_pred_2020 = ep_test_t1.apply(lambda row: params_ep_model_t1['intercept'] + \
    params_ep_model_t1['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + params_ep_model_t1['Neg_E_interaction_term']*
                          row['Neg_E_interaction_term'] + \
    params_ep_model_t1['EPS']*row['EPS'] , axis=1)


# Reset the index of the y_pred_2020 DataFrame
y_pred_2020 = y_pred_2020.reset_index()

y_true_2020 = test_data_2020[['gvkey', 'E_t']]
merged_df = y_true_2020.merge(y_pred_2020, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2020
mae_2020 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2020:", mae_2020)
print("MSE for 2020:", mse_2020)
print("RMSE for 2020:", rmse_2020)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2020: 295.23532015775567
MSE for 2020: 861015.1679997757
RMSE for 2020: 927.9090300238357
Mean bias: -200.68666330344473
Median bias: 1.1771928998407724
p-value for mean bias: 3.477584868901503e-52
p-value for median bias: 9.172985969485354e-53
Mean accuracy: 1.1946360443241164
Median accuracy: 1.6770041852323767
p-value for mean accuracy: 0.5033637517255101
p-value for median accuracy: 0.09725647919764034
t-statistic: -15.385774169189597
p-value for t-statistic: 3.477584868901503e-52


#### EP model for earnings in year E_t+2

In [129]:
ep_train_t2 = train_data[['dummy_Neg_E_t','Neg_E_interaction_term','EPS','E_t2_EP_RI']].dropna()

# Estimate coefficients for HVZ Model at industry and fyear level
ep_model_t2 = ols(ep_train_t2,'E_t2_EP_RI',['dummy_Neg_E_t','Neg_E_interaction_term','EPS'])
params_ep_model_t2 = ep_model_t2.params
ep_model_t2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t2_EP_RI   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                 2.320e+04
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        18:55:07   Log-Likelihood:            -3.7320e+05
No. Observations:               44116   AIC:                         7.464e+05
Df Residuals:                   44112   BIC:                         7.464e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
dummy_Neg_E_t             -6.9388     11.759     -0.590      0.555     -29.986      16.109
Neg_E_interaction_term     7.0852      8.747      0.810      0.418     -10.059      24.230
EPS                       -1.4030      0.005   -263.760      0.000      -1.413      -1.393
intercept                  9.4540      7.485      1.263      0.207      -5.217      24.125
==============================================================================
Omnibus:                    87905.342   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     744931879219.128
Skew:                          13.780   Prob(JB):                         0.00
Kurtosis:                   20134.004   Cond. No.                     2.51e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.51e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [130]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
ep_test_t2 = test_data_2019[['gvkey','dummy_Neg_E_t','Neg_E_interaction_term','EPS','E_t2_EP_RI']].dropna()
ep_test_t2 = ep_test_t2.set_index('gvkey')

# Make predictions using the model for 2022 test data
y_pred_2021 = ep_test_t2.apply(lambda row: params_ep_model_t2['intercept'] + \
    params_ep_model_t2['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + params_ep_model_t2['Neg_E_interaction_term']*
                          row['Neg_E_interaction_term'] + \
    params_ep_model_t2['EPS']*row['EPS'] , axis=1)


# Reset the index of the y_pred_2021 DataFrame
y_pred_2021 = y_pred_2021.reset_index()

y_true_2021 = test_data_2021[['gvkey', 'E_t']]
merged_df = y_true_2021.merge(y_pred_2021, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2021
mae_2021 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2021:", mae_2021)
print("MSE for 2021:", mse_2021)
print("RMSE for 2021:", rmse_2021)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2021: 421.09210339326825
MSE for 2021: 1818009.3094342707
RMSE for 2021: 1348.3357554534666
Mean bias: -367.6133715565165
Median bias: 2.43719581950836
p-value for mean bias: 1.3404978403174376e-77
p-value for median bias: 1.4399836432002398e-78
Mean accuracy: 1.7391316057708062
Median accuracy: 0.8731841620020917
p-value for mean accuracy: 0.40663593970011447
p-value for median accuracy: 0.33095536751763865
t-statistic: -19.01379457311935
p-value for t-statistic: 1.3404978403174376e-77


#### EP model for earnings in year E_t+3

In [131]:
ep_train_t3 = train_data[['dummy_Neg_E_t','Neg_E_interaction_term','EPS','E_t3_EP_RI']].dropna()

# Estimate coefficients for HVZ Model at industry and fyear level
ep_model_t3 = ols(ep_train_t3,'E_t3_EP_RI',['dummy_Neg_E_t','Neg_E_interaction_term','EPS'])
params_ep_model_t3 = ep_model_t3.params
ep_model_t3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t3_EP_RI   R-squared:                       0.483
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                 1.262e+04
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        18:58:47   Log-Likelihood:            -3.6906e+05
No. Observations:               40535   AIC:                         7.381e+05
Df Residuals:                   40531   BIC:                         7.382e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
dummy_Neg_E_t             -8.2466     23.483     -0.351      0.725     -54.274      37.781
Neg_E_interaction_term    12.1292     17.780      0.682      0.495     -22.720      46.978
EPS                       -1.9722      0.010   -194.499      0.000      -1.992      -1.952
intercept                 13.3827     14.720      0.909      0.363     -15.469      42.234
==============================================================================
Omnibus:                    71892.687   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     611441835820.766
Skew:                          10.297   Prob(JB):                         0.00
Kurtosis:                   19029.892   Cond. No.                     2.63e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [132]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
ep_test_t3 = test_data_2019[['gvkey','dummy_Neg_E_t','Neg_E_interaction_term','EPS','E_t3_EP_RI']].dropna()
ep_test_t3 = ep_test_t3.set_index('gvkey')

# Make predictions using the model for 2022 test data
y_pred_2022 = ep_test_t3.apply(lambda row: params_ep_model_t3['intercept'] + \
    params_ep_model_t3['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + params_ep_model_t3['Neg_E_interaction_term']*
                          row['Neg_E_interaction_term'] + \
    params_ep_model_t3['EPS']*row['EPS'] , axis=1)


# Reset the index of the y_pred_2022 DataFrame
y_pred_2022 = y_pred_2022.reset_index()

y_true_2022 = test_data_2022[['gvkey', 'E_t']]
merged_df = y_true_2022.merge(y_pred_2022, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2022
mae_2022 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2022:", mae_2022)
print("MSE for 2022:", mse_2022)
print("RMSE for 2022:", rmse_2022)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2022: 524.2327423258337
MSE for 2022: 2561296.5190807614
RMSE for 2022: 1600.4051109268432
Mean bias: -462.2586372168261
Median bias: 0.5445984292695928
p-value for mean bias: 3.5756259813859655e-65
p-value for median bias: 2.5726075320509307e-65
Mean accuracy: -2.756425962717368
Median accuracy: 0.43693321723349027
p-value for mean accuracy: 0.3798912425558185
p-value for median accuracy: 0.45537329128563475
t-statistic: -17.42809190447095
p-value for t-statistic: 3.5756259813859655e-65


## Residual Income Model

#### RI model for earnings in year E_t+1

In [133]:
ri_train_t1 = train_data[['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t1_EP_RI']].dropna()

# Estimate coefficients for HVZ Model at industry and fyear level
ri_model_t1 = ols(ri_train_t1,'E_t1_EP_RI',['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t'])
params_ri_model_t1 = ri_model_t1.params
ri_model_t1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t1_EP_RI   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     21.40
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.18e-17
Time:                        19:01:45   Log-Likelihood:            -3.3787e+05
No. Observations:               48142   AIC:                         6.757e+05
Df Residuals:                   48137   BIC:                         6.758e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
dummy_Neg_E_t              6.7558      3.041      2.222      0.026       0.795      12.716
Neg_E_interaction_term    -1.7362      1.960     -0.886      0.376      -5.578       2.105
B_t                        0.4533      0.377      1.201      0.230      -0.286       1.193
TACC_t                     0.7337      0.396      1.853      0.064      -0.042       1.510
intercept                 -8.7165      2.358     -3.696      0.000     -13.338      -4.095
==============================================================================
Omnibus:                   223502.387   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    1736923425898.895
Skew:                         167.027   Prob(JB):                         0.00
Kurtosis:                   29427.284   Cond. No.                         53.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [134]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
ri_test_t1 = test_data_2019[['gvkey','dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t1_EP_RI']].dropna()
ri_test_t1 = ri_test_t1.set_index('gvkey')

# Make predictions using the model
y_pred_2020 = ri_test_t1.apply(lambda row: params_ri_model_t1['intercept'] + \
    params_ri_model_t1['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + params_ri_model_t1['Neg_E_interaction_term']*
                          row['Neg_E_interaction_term'] + \
    params_ri_model_t1['B_t']*row['B_t'] + \
    params_ri_model_t1['TACC_t']*row['TACC_t'] , axis=1)


# Reset the index of the y_pred_2022 DataFrame
y_pred_2020 = y_pred_2020.reset_index()

y_true_2020 = test_data_2020[['gvkey', 'E_t']]
merged_df = y_true_2020.merge(y_pred_2020, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2020
mae_2020 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2020:", mae_2020)
print("MSE for 2020:", mse_2020)
print("RMSE for 2020:", rmse_2020)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2020: 294.22782156147224
MSE for 2020: 855418.7495544594
RMSE for 2020: 924.8885065533356
Mean bias: -198.27634614366502
Median bias: -0.7830144558256233
p-value for mean bias: 2.7781508722465745e-51
p-value for median bias: 6.682443660692059e-51
Mean accuracy: 0.7931386642552858
Median accuracy: 1.4423164505642112
p-value for mean accuracy: 0.7555694405794198
p-value for median accuracy: 0.3286189118690286
t-statistic: -15.244088922089777
p-value for t-statistic: 2.7781508722465745e-51


#### RI model for earnings in year E_t+2

In [135]:
ri_train_t2 = train_data[['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t2_EP_RI']].dropna()

# Estimate coefficients for HVZ Model at industry and fyear level
ri_model_t2 = ols(ri_train_t2,'E_t2_EP_RI',['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t'])
params_ri_model_t2 = ri_model_t2.params
ri_model_t2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t2_EP_RI   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     12.06
Date:                Sat, 06 May 2023   Prob (F-statistic):           8.47e-10
Time:                        19:06:00   Log-Likelihood:            -3.9407e+05
No. Observations:               44116   AIC:                         7.881e+05
Df Residuals:                   44111   BIC:                         7.882e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
dummy_Neg_E_t             42.7213     21.623      1.976      0.048       0.341      85.102
Neg_E_interaction_term   -47.9389     14.217     -3.372      0.001     -75.805     -20.073
B_t                        1.8022      2.674      0.674      0.500      -3.439       7.044
TACC_t                     3.3758      2.805      1.203      0.229      -2.123       8.875
intercept                -55.3404     16.642     -3.325      0.001     -87.958     -22.722
==============================================================================
Omnibus:                   216755.339   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    3245045687980.930
Skew:                         203.018   Prob(JB):                         0.00
Kurtosis:                   42017.353   Cond. No.                         54.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
ri_test_t2 = test_data_2019[['gvkey','dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t2_EP_RI']].dropna()
ri_test_t2 = ri_test_t2.set_index('gvkey')

# Make predictions using the model
y_pred_2021 = ri_test_t2.apply(lambda row: params_ri_model_t2['intercept'] + \
    params_ri_model_t2['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + params_ri_model_t2['Neg_E_interaction_term']*
                          row['Neg_E_interaction_term'] + \
    params_ri_model_t2['B_t']*row['B_t'] + \
    params_ri_model_t2['TACC_t']*row['TACC_t'] , axis=1)


# Reset the index of the y_pred_2022 DataFrame
y_pred_2021 = y_pred_2021.reset_index()

y_true_2021= test_data_2021[['gvkey', 'E_t']]
merged_df = y_true_2021.merge(y_pred_2021, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2021
mae_2021 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2021:", mae_2021)
print("MSE for 2021:", mse_2021)
print("RMSE for 2021:", rmse_2021)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2021: 433.77938678199183
MSE for 2021: 1775516.3785760547
RMSE for 2021: 1332.4850387813196
Mean bias: -341.36490886900157
Median bias: -10.808258092113311
p-value for mean bias: 2.0027237785518484e-68
p-value for median bias: 2.096175152511329e-64
Mean accuracy: 4.0687597545757495
Median accuracy: 0.8496381371489664
p-value for mean accuracy: 0.5170994476980539
p-value for median accuracy: 0.4967810992762579
t-statistic: -17.782802718672016
p-value for t-statistic: 2.0027237785518484e-68


#### RI model for earnings in year E_t+3

In [138]:
ri_train_t3 = train_data[['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t3_EP_RI']].dropna()

# Estimate coefficients for HVZ Model at industry and fyear level
ri_model_t3 = ols(ri_train_t3,'E_t3_EP_RI',['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t'])
params_ri_model_t3 = ri_model_t3.params
ri_model_t3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t3_EP_RI   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     9.202
Date:                Sat, 06 May 2023   Prob (F-statistic):           1.99e-07
Time:                        19:12:16   Log-Likelihood:            -3.8241e+05
No. Observations:               40535   AIC:                         7.648e+05
Df Residuals:                   40530   BIC:                         7.649e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
dummy_Neg_E_t             64.3855     37.380      1.722      0.085      -8.880     137.651
Neg_E_interaction_term   -75.1249     25.030     -3.001      0.003    -124.184     -26.066
B_t                        2.7940      4.588      0.609      0.543      -6.198      11.786
TACC_t                     4.8003      4.817      0.997      0.319      -4.641      14.242
intercept                -83.3170     28.529     -2.920      0.003    -139.234     -27.400
==============================================================================
Omnibus:                   198050.230   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):    2688593287810.559
Skew:                         199.070   Prob(JB):                         0.00
Kurtosis:                   39899.194   Cond. No.                         55.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [139]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp

# Select the same columns as used in training the model
ri_test_t3 = test_data_2019[['gvkey','dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t3_EP_RI']].dropna()
ri_test_t3 = ri_test_t3.set_index('gvkey')

# Make predictions using the model
y_pred_2022 = ri_test_t3.apply(lambda row: params_ri_model_t3['intercept'] + \
    params_ri_model_t3['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + params_ri_model_t3['Neg_E_interaction_term']*
                          row['Neg_E_interaction_term'] + \
    params_ri_model_t3['B_t']*row['B_t'] + \
    params_ri_model_t3['TACC_t']*row['TACC_t'] , axis=1)


# Reset the index of the y_pred_2022 DataFrame
y_pred_2022 = y_pred_2022.reset_index()

y_true_2022= test_data_2022[['gvkey', 'E_t']]
merged_df = y_true_2022.merge(y_pred_2022, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2022
mae_2022 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2022:", mae_2022)
print("MSE for 2022:", mse_2022)
print("RMSE for 2022:", rmse_2022)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2022: 544.6371003295816
MSE for 2022: 2496224.0517631797
RMSE for 2022: 1579.9443191970975
Mean bias: -415.3672316693624
Median bias: -16.710498936850897
p-value for mean bias: 6.617141133965427e-54
p-value for median bias: 6.2612782721511795e-50
Mean accuracy: -26.09587845236394
Median accuracy: 0.5735684813266599
p-value for mean accuracy: 0.3076445222523254
p-value for median accuracy: 0.3153188337491373
t-statistic: -15.74058454532862
p-value for t-statistic: 6.617141133965427e-54


#### Based on the best model, I have further improved the model by considering additional factors. Additional factors can include the most recent analyst forecast (released prior to earnings), sales growth, return on assets, research and development expenses, financial ratios, and dividend payout ratio,

Selected Best Model is HVZ for predicting earnings for 2020 and 2021. 
Selected Best Model is RI for predicting earnings for 2022.

Now we will create these additional predictors to augment our original models in an attempt to improve our model forecast accruacy.

#### Additional Predictors

1. Total Debt, df['total_debt'] = df['dltt']+df['dlc']
2. Dividend payout ratio, 'DVC/NI'
3. Sales growth, df['SALE'] = (df['SALE'] - df['SALE'].shift(1)) / df['SALE'].shift(1)
4. Return on assets, 'NI'/'AT'
5. Debt-to-equity ratio, total_debt / 'LSE'
6. Cash flow from operations, 'OANCF'
7. Research and development expenses, 'RDIP'


In [140]:
df_new = merged_data[['gvkey','tic','conm','invch','sic','prcc_f','fyear','cusip', 'industry',
                      'dltt','dlc','dvc','ni','sale','at','lse','oancf','rdip']]

In [141]:
# define a function to calculate new features within each group
def cal_new_features(group):
    group['total_debt'] = group['dltt']+group['dlc']
    group['div_payout_ratio'] = group['dvc']/group['ni']
    group['sale'] = (group['sale'] - group['sale'].shift(1))/group['sale'].shift(1)
    group['roa'] = group['ni']/group['sale']
    group['deq_ratio'] = group['total_debt']/group['lse']
    return group

In [142]:
# Fill all remaining None Type values with zero
df_new.fillna(0, inplace=True)

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\1339287206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.fillna(0, inplace=True)


In [143]:
# This is to make sure that the data we use is for one company only and that it doesnot take values from a row above that belongs to some other company
df_new = df_new.groupby('gvkey')

In [144]:
new_features_df = df_new.apply(cal_new_features)

In [145]:
df_features = delta_df.merge(new_features_df, on=['tic','fyear'], how='left')

In [146]:
rename_dict = {}
for col in df_features.columns:
    if col.endswith('_x'):
        rename_dict[col] = col[:-2]
# rename the columns
df_features = df_features.rename(columns=rename_dict)

In [149]:
# Create an empty data frame to hold the winsorized data
df_win_new = pd.DataFrame()

# List of columns to winsorize
cols_to_winsorize = ['at','dvc','E_t','dummy_Neg_E_t','ACT','dltt','dlc','dvc','ni','sale',
                     'lse','oancf','rdip','invch','E_t1_HVZ','E_t2_HVZ','E_t3_EP_RI','Neg_E_interaction_term','B_t','TACC_t',
                     'div_payout_ratio','total_debt','deq_ratio','roa']

# Winsorize by year
for year in df_features['fyear'].unique():
    # Subset the data for the current year
    df_year = df_features[df_features['fyear'] == year]
    print(df_year.shape[0])
    # Winsorize the columns for the current year's data
    for col in cols_to_winsorize:
        #df_year[col].dropna(inplace=True)
        #df_year[col] = winsorize(df_year[col], limits=(0.01, 0.01))
        # Apply winsorize function to each column of the DataFrame
        df_year[col] = df_year[col].apply(lambda x: winsorize(x, limits=(0.01, 0.01)))
    
    # Append the winsorized data for the current year to df_winsorized
    df_win_new = pd.concat([df_win_new, df_year])

5789


C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2029543419.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = df_year[col].apply(lambda x: winsorize(x, limits=(0.01, 0.01)))


5776


C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2029543419.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = df_year[col].apply(lambda x: winsorize(x, limits=(0.01, 0.01)))


6107


...

C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2029543419.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = df_year[col].apply(lambda x: winsorize(x, limits=(0.01, 0.01)))


5635


C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2029543419.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = df_year[col].apply(lambda x: winsorize(x, limits=(0.01, 0.01)))


4404


C:\Users\abbas\AppData\Local\Temp\ipykernel_32860\2029543419.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[col] = df_year[col].apply(lambda x: winsorize(x, limits=(0.01, 0.01)))


In [150]:
df_win_new

,gvkey,tic,conm,invch,sic,prcc_f,fyear,cusip,industry,ib,...,ni,sale,at_y,lse,oancf,rdip,total_debt,div_payout_ratio,roa,deq_ratio
0,001004,AIR,AAR CORP,-34.615,5080,26.390,2010.0,000361105,50,73.139,...,69.826,0.313302,1703.727,1703.727,108.598,0.0,443.877,0.042720,222.871546,0.260533
12,001013,ADCT.1,ADC TELECOMMUNICATIONS INC,13.800,3661,12.670,2010.0,000886309,36,77.200,...,62.000,0.160429,1474.500,1474.500,136.600,0.0,651.100,0.000000,386.462789,0.441573
13,001019,AFAP,AFA PROTECTIVE SYSTEMS INC,0.260,7380,240.000,2010.0,001038108,73,0.997,...,0.997,-0.072240,29.546,29.546,0.829,0.0,8.653,7.993982,-13.801202,0.292865
24,001045,AAL,AMERICAN AIRLINES GROUP INC,-81.000,4512,7.790,2010.0,02376R102,45,-471.000,...,-471.000,0.113119,25088.000,25088.000,1241.000,0.0,11136.000,-0.000000,-4163.740346,0.443878
27,001050,CECO,CECO ENVIRONMENTAL CORP,2.201,3564,5.960,2010.0,125141101,35,2.305,...,2.105,0.011634,74.791,74.791,2.938,0.0,10.800,0.000000,180.929762,0.144402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73157,349972,INDP,INDAPTUS THERAPEUTICS INC,0.000,2836,1.450,2022.0,45339J105,28,-14.323,...,-14.323,NaN,28.064,28.064,-13.078,0.0,0.080,-0.000000,NaN,0.002851
73159,351038,QNRX,CELLECT BIOTECHNOLOGY LTD,0.000,2834,1.420,2022.0,74907L201,28,-9.381,...,-9.381,NaN,14.458,14.458,-8.481,0.0,0.000,-0.000000,NaN,0.000000
73160,351491,IVCGF,IVECO GROUP N V,-234.055,3711,5.880,2022.0,N47017103,37,157.105,...,157.105,0.066417,17113.769,17113.769,1503.721,0.0,4737.734,0.000000,2365.444075,0.276838
73162,351590,DTRUY,DAIMLER TRUCK HOLDING AG,-1221.573,3713,15.446,2022.0,23384L101,37,2848.198,...,2848.198,0.201161,68366.372,68366.372,-558.952,0.0,22271.521,0.000000,14158.828967,0.325767


In [151]:
# Training dataset = 2010-2019 Test dataset = 2020, 2021, and 2022
# split data into training and test datasets based on date
train_data_new = df_win_new[(df_win_new['fyear'] >= 2010) & (df_win_new['fyear'] <= 2018)]
test_data_new = df_win_new[df_win_new['fyear'].isin([2019,2020, 2021, 2022])]
test_data_new_2019 = df_win_new[df_win_new['fyear'] == 2019]
test_data_new_2020 = df_win_new[df_win_new['fyear'] == 2020]
test_data_new_2021 = df_win_new[df_win_new['fyear'] == 2021]
test_data_new_2022 = df_win_new[df_win_new['fyear'] == 2022]

#### Modified Model for Year 2020 with new features using HVZ model selected in Q1

In [156]:
# Remove rows with infinite values
cols_to_check_for_inf = ['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale','E_t1_HVZ']
hvz_train_new_t1 = train_data_new[cols_to_check_for_inf].replace([np.inf, -np.inf], np.nan)

hvz_train_new_t1 = hvz_train_new_t1.dropna(subset=['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale','E_t1_HVZ'])


# Estimate coefficients for HVZ Model at industry and fyear level
hvz_model_new_t1 = ols(hvz_train_new_t1,'E_t1_HVZ',['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale'])
params_hvz_model_new_t1 = hvz_model_new_t1.params
hvz_model_new_t1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               E_t1_HVZ   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                 1.236e+04
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        19:44:43   Log-Likelihood:            -2.6291e+05
No. Observations:               32964   AIC:                         5.258e+05
Df Residuals:                   32950   BIC:                         5.260e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
at                  -0.0026      0.001     -4.244      0.000      -0.004      -0.001
dvc                  0.0461      0.008      5.852      0.000       0.031       0.061
E_t                  0.6155      0.006    105.912      0.000       0.604       0.627
dummy_Neg_E_t       -4.8699      8.151     -0.597      0.550     -20.847      11.107
ACT                  0.1626      0.006     26.873      0.000       0.151       0.174
div_payout_ratio     0.1124      0.432      0.260      0.795      -0.734       0.959
total_debt           0.0192      0.001     15.198      0.000       0.017       0.022
deq_ratio           -0.0356      0.282     -0.126      0.900      -0.589       0.518
roa              -3.041e-05   6.64e-06     -4.579      0.000   -4.34e-05   -1.74e-05
oancf                0.2233      0.005     46.147      0.000       0.214       0.233
rdip                -0.3787      0.085     -4.453      0.000      -0.545      -0.212
invch               -0.2633      0.015    -17.844      0.000      -0.292      -0.234
sale                -0.0046      0.042     -0.109      0.913      -0.088       0.078
intercept            9.0840      5.164      1.759      0.079      -1.039      19.207
==============================================================================
Omnibus:                    17735.712   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        123644356.023
Skew:                          -0.725   Prob(JB):                         0.00
Kurtosis:                     303.032   Cond. No.                     1.36e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.36e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [158]:
# Testing Accuracy on out of sample data
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp
# Select the same columns as used in training the model
# Remove rows with infinite values
hvz_test_new_t1 = test_data_new_2019[cols_to_check_for_inf+['gvkey']].replace([np.inf, -np.inf], np.nan)
hvz_test_new_t1 = hvz_test_new_t1.dropna(subset=cols_to_check_for_inf)
hvz_test_new_t1 = hvz_test_new_t1.set_index('gvkey')


# Make predictions using the model for 2020 test data
y_pred_2020_new = hvz_test_new_t1.apply(lambda row: params_hvz_model_new_t1['intercept'] + \
    params_hvz_model_new_t1['at']*row['at'] + params_hvz_model_new_t1['dvc']*row['dvc'] + \
    params_hvz_model_new_t1['E_t']*row['E_t'] + params_hvz_model_new_t1['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_new_t1['ACT']*row['ACT'] + params_hvz_model_new_t1['div_payout_ratio']*row['div_payout_ratio'] + \
    params_hvz_model_new_t1['total_debt']*row['total_debt'] + params_hvz_model_new_t1['deq_ratio']*row['deq_ratio'] + \
    params_hvz_model_new_t1['roa']*row['roa'] + params_hvz_model_new_t1['oancf']*row['oancf'] + \
    params_hvz_model_new_t1['rdip']*row['rdip'] + params_hvz_model_new_t1['invch']*row['invch'] + \
    params_hvz_model_new_t1['sale']*row['sale'], axis=1)

# Reset the index of the y_pred_2020 DataFrame
y_pred_2020_new= y_pred_2020_new.reset_index()
y_pred_2020_new
y_true_2020_new = test_data_new_2020[['gvkey', 'E_t']]
merged_df = y_true_2020_new.merge(y_pred_2020_new, on='gvkey')
# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2020
mae_2020 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2020 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2020:", mae_2020)
print("MSE for 2020:", mse_2020)
print("RMSE for 2020:", rmse_2020)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()
from scipy.stats import ttest_rel
# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2020: 274.0529878040788
MSE for 2020: 1944398.1433454952
RMSE for 2020: 1394.4167753385268
Mean bias: 98.9650596558048
Median bias: 8.683123700096889
p-value for mean bias: 1.1574534767716017e-05
p-value for median bias: 6.293644048016392e-05
Mean accuracy: 1.473281105558993
Median accuracy: 0.9534771924018002
p-value for mean accuracy: 0.5850816817413258
p-value for median accuracy: 0.5487439250773191
t-statistic: 4.39128202090691
p-value for t-statistic: 1.1574534767716017e-05


#### Modified Model for Year 2021 to predict for 2021 earnings with new features using HVZ model selected in Q1

In [159]:
# Remove rows with infinite values
cols_to_check_for_inf = ['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale','E_t2_HVZ']
hvz_train_new_t2 = train_data_new[cols_to_check_for_inf].replace([np.inf, -np.inf], np.nan)

hvz_train_new_t2 = hvz_train_new_t2.dropna(subset=['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale','E_t2_HVZ'])


# Estimate coefficients for HVZ Model at industry and fyear level
hvz_model_new_t2 = ols(hvz_train_new_t2,'E_t2_HVZ',['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale'])
params_hvz_model_new_t2 = hvz_model_new_t2.params
hvz_model_new_t2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               E_t2_HVZ   R-squared:                       0.679
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     4934.
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        20:00:41   Log-Likelihood:            -2.5364e+05
No. Observations:               30391   AIC:                         5.073e+05
Df Residuals:                   30377   BIC:                         5.074e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
at                  -0.0138      0.001    -15.266      0.000      -0.016      -0.012
dvc                  0.0290      0.011      2.532      0.011       0.007       0.051
E_t                  0.4412      0.008     52.199      0.000       0.425       0.458
dummy_Neg_E_t      -17.6157     12.378     -1.423      0.155     -41.878       6.647
ACT                  0.2255      0.009     25.604      0.000       0.208       0.243
div_payout_ratio     0.3111      0.648      0.480      0.631      -0.960       1.582
total_debt           0.0498      0.002     27.028      0.000       0.046       0.053
deq_ratio           -0.0844      0.465     -0.182      0.856      -0.995       0.826
roa              -5.185e-05   9.62e-06     -5.390      0.000   -7.07e-05    -3.3e-05
oancf                0.3405      0.007     48.389      0.000       0.327       0.354
rdip                -1.1376      0.126     -9.012      0.000      -1.385      -0.890
invch               -0.3023      0.022    -13.897      0.000      -0.345      -0.260
sale                -0.0101      0.062     -0.164      0.870      -0.131       0.111
intercept           32.2529      7.715      4.180      0.000      17.131      47.375
==============================================================================
Omnibus:                    35448.761   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        114743648.049
Skew:                          -4.978   Prob(JB):                         0.00
Kurtosis:                     303.857   Cond. No.                     1.41e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.41e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [160]:
# Testing Accuracy on out of sample data
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp
# Select the same columns as used in training the model
# Remove rows with infinite values
hvz_test_new_t2 = test_data_new_2019[cols_to_check_for_inf+['gvkey']].replace([np.inf, -np.inf], np.nan)
hvz_test_new_t2 = hvz_test_new_t2.dropna(subset=cols_to_check_for_inf)
hvz_test_new_t2 = hvz_test_new_t2.set_index('gvkey')

# Make predictions using the model for 2021 test data
y_pred_2021_new = hvz_test_new_t2.apply(lambda row: params_hvz_model_new_t2['intercept'] + \
    params_hvz_model_new_t2['at']*row['at'] + params_hvz_model_new_t2['dvc']*row['dvc'] + \
    params_hvz_model_new_t2['E_t']*row['E_t'] + params_hvz_model_new_t2['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_new_t2['ACT']*row['ACT'] + params_hvz_model_new_t2['div_payout_ratio']*row['div_payout_ratio'] + \
    params_hvz_model_new_t2['total_debt']*row['total_debt'] + params_hvz_model_new_t2['deq_ratio']*row['deq_ratio'] + \
    params_hvz_model_new_t2['roa']*row['roa'] + params_hvz_model_new_t2['oancf']*row['oancf'] + \
    params_hvz_model_new_t2['rdip']*row['rdip'] + params_hvz_model_new_t2['invch']*row['invch'] + \
    params_hvz_model_new_t2['sale']*row['sale'], axis=1)


# Reset the index of the y_pred_2020 DataFrame
y_pred_2021_new= y_pred_2021_new.reset_index()
y_pred_2021_new
y_true_2021_new = test_data_new_2021[['gvkey', 'E_t']]
merged_df = y_true_2021_new.merge(y_pred_2021_new, on='gvkey')
# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2021
mae_2021 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_2021 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2021:", mae_2021)
print("MSE for 2021:", mse_2021)
print("RMSE for 2021:", rmse_2021)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()
from scipy.stats import ttest_rel
# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2021: 339.69354764249016
MSE for 2021: 2430544.9862655387
RMSE for 2021: 1559.0205214382327
Mean bias: -160.88999876956558
Median bias: 16.451239113362583
p-value for mean bias: 6.150709902935531e-10
p-value for median bias: 9.413576572260152e-12
Mean accuracy: -0.8152329365986413
Median accuracy: 0.8236778920953923
p-value for mean accuracy: 0.6816736493935621
p-value for median accuracy: 0.711130353587111
t-statistic: -6.203549993425207
p-value for t-statistic: 6.150709902935531e-10


#### Modified Model for Year 2022 to predict for 2022 earnings with new features using RI model selected in Q1

In [180]:
# Remove rows with infinite values
cols_to_check_for_inf = ['dummy_Neg_E_t','Neg_E_interaction_term','B_t','TACC_t','E_t3_EP_RI','at','dvc',
                         'div_payout_ratio','total_debt','deq_ratio','roa','oancf','rdip','invch','sale' ]

ri_train_new_t3 = train_data_new[cols_to_check_for_inf].replace([np.inf, -np.inf], np.nan)
# remove NANs 
ri_train_new_t3 = ri_train_new_t3.dropna(subset=cols_to_check_for_inf)


# Estimate coefficients for HVZ Model at industry and fyear level
ri_model_new_t3 = ols(ri_train_new_t3,'E_t3_EP_RI',['at','dvc','dummy_Neg_E_t','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale',
                                                     'Neg_E_interaction_term','B_t','TACC_t'])
params_ri_model_new_t3 = ri_model_new_t3.params
ri_model_new_t3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             E_t3_EP_RI   R-squared:                       0.964
Model:                            OLS   Adj. R-squared:                  0.964
Method:                 Least Squares   F-statistic:                 6.357e+04
Date:                Sat, 06 May 2023   Prob (F-statistic):               0.00
Time:                        23:42:31   Log-Likelihood:            -2.6872e+05
No. Observations:               33172   AIC:                         5.375e+05
Df Residuals:                   33157   BIC:                         5.376e+05
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
at                        -0.0013      0.001     -2.007      0.045      -0.003      -3e-05
dvc                        0.0070      0.008      0.848      0.396      -0.009       0.023
dummy_Neg_E_t             30.4566      9.281      3.282      0.001      12.266      48.647
div_payout_ratio          -0.0574      0.414     -0.139      0.890      -0.868       0.754
total_debt                 0.0016      0.001      1.149      0.250      -0.001       0.004
deq_ratio                 -0.0224      0.231     -0.097      0.923      -0.476       0.431
roa                     1.146e-06   7.46e-06      0.154      0.878   -1.35e-05    1.58e-05
oancf                      0.0028      0.004      0.771      0.441      -0.004       0.010
rdip                      -0.0196      0.096     -0.204      0.839      -0.209       0.169
invch                     -0.0144      0.016     -0.903      0.366      -0.046       0.017
sale                       0.0120      0.049      0.246      0.806      -0.083       0.107
Neg_E_interaction_term     4.0722      0.006    652.807      0.000       4.060       4.084
B_t                      -18.2668      0.065   -282.065      0.000     -18.394     -18.140
TACC_t                    22.1315      0.075    297.002      0.000      21.985      22.278
intercept                 -7.7121      5.719     -1.348      0.178     -18.922       3.497
==============================================================================
Omnibus:                   123346.961   Durbin-Watson:                   1.983
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     288660011917.089
Skew:                          80.745   Prob(JB):                         0.00
Kurtosis:                   14453.604   Cond. No.                     1.35e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.35e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [182]:
# Testing Accuracy on out of sample data
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import ttest_1samp
# Select the same columns as used in training the model
# Remove rows with infinite values
ri_test_new_t3 = test_data_new_2019[cols_to_check_for_inf+['gvkey']].replace([np.inf, -np.inf], np.nan)
ri_test_new_t3 = ri_test_new_t3.dropna(subset=cols_to_check_for_inf)
ri_test_new_t3 = ri_test_new_t3.set_index('gvkey')

# Make predictions using the model
y_pred_new_2022 = ri_test_new_t3.apply(lambda row: params_ri_model_new_t3['intercept'] + \
    params_ri_model_new_t3['at']*row['at'] + \
    params_ri_model_new_t3['dvc']*row['dvc'] + \
    params_ri_model_new_t3['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_ri_model_new_t3['div_payout_ratio']*row['div_payout_ratio'] + \
    params_ri_model_new_t3['total_debt']*row['total_debt'] + \
    params_ri_model_new_t3['deq_ratio']*row['deq_ratio'] + \
    params_ri_model_new_t3['roa']*row['roa'] + \
    params_ri_model_new_t3['oancf']*row['oancf'] + \
    params_ri_model_new_t3['rdip']*row['rdip'] + \
    params_ri_model_new_t3['invch']*row['invch'] + \
    params_ri_model_new_t3['sale']*row['sale'] + \
    params_ri_model_new_t3['Neg_E_interaction_term']*row['Neg_E_interaction_term'] + \
    params_ri_model_new_t3['B_t']*row['B_t'] + \
    params_ri_model_new_t3['TACC_t']*row['TACC_t'], axis=1)

# Reset the index of the y_pred_2022 DataFrame
y_pred_new_2022 = y_pred_new_2022.reset_index()

y_true_new_2022= test_data_new_2022[['gvkey', 'E_t']]
merged_df = y_true_new_2022.merge(y_pred_new_2022, on='gvkey')

# Rename the predicted column
merged_df = merged_df.rename(columns={0: 'predicted'})

# Calculate accuracy for 2022
mae_new_2022 = mean_absolute_error(merged_df['E_t'], merged_df['predicted'])
mse_new_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'])
rmse_new_2022 = mean_squared_error(merged_df['E_t'], merged_df['predicted'], squared=False)

print("MAE for 2022:", mae_new_2022)
print("MSE for 2022:", mse_new_2022)
print("RMSE for 2022:", rmse_new_2022)

# Calculate bias and accuracy
y_true = merged_df['E_t']
y_pred = merged_df['predicted']
bias = y_pred - y_true
accuracy = 1 - abs(bias) / (y_true + 1e-9)

# Compute mean and median of bias and accuracy
bias_mean = bias.mean()
bias_median = bias.median()
accuracy_mean = accuracy.mean()
accuracy_median = accuracy.median()

# Compute p-values for mean and median bias and accuracy
_, pval_bias_mean = ttest_1samp(bias, 0)
_, pval_bias_median = ttest_1samp(bias, bias_median)
_, pval_accuracy_mean = ttest_1samp(accuracy, 1)
_, pval_accuracy_median = ttest_1samp(accuracy, accuracy_median)

# Compute t-statistic
t_statistic, p_value = ttest_rel(y_pred, y_true)

# Print results
print(f"Mean bias: {bias_mean}")
print(f"Median bias: {bias_median}")
print(f"p-value for mean bias: {pval_bias_mean}")
print(f"p-value for median bias: {pval_bias_median}")
print(f"Mean accuracy: {accuracy_mean}")
print(f"Median accuracy: {accuracy_median}")
print(f"p-value for mean accuracy: {pval_accuracy_mean}")
print(f"p-value for median accuracy: {pval_accuracy_median}")
print(f"t-statistic: {t_statistic}")
print(f"p-value for t-statistic: {p_value}")

MAE for 2022: 830.6032446014058
MSE for 2022: 13953025.592651855
RMSE for 2022: 3735.3748931870086
Mean bias: -722.003073426322
Median bias: -27.805979775128293
p-value for mean bias: 2.257948183452738e-25
p-value for median bias: 1.3075694405887213e-23
Mean accuracy: -2.23480181811288
Median accuracy: 0.1542689448828456
p-value for mean accuracy: 0.3436131979315179
p-value for median accuracy: 0.48425755185535757
t-statistic: -10.509702000185818
p-value for t-statistic: 2.257948183452738e-25


#### Based on the coefficients I have estimated above, I have forecasted 2020 annual earnings per share (EPS) for 10 stocks picked on the basis of: business, industry, and information environment; These forecasts are fairly accurate (comparing forecasted earnings to actual earnings). I have also compared my forecasts with analysts’ forecasts - IBES (comparing forecasted earnings to the analysts’ forecast for the period). The mean absolute error comes out to be 0.0473 and the Mean Squared Error comes out to be 0.0094.

In [223]:
# Merging IBES dataset with the Ticker List
# Importing Link File
# Set the file path
file_path = r'C:\Users\abbas\OneDrive\Desktop\MMA Studies\Finance and Accounting Insights\Group Project\Link_File.xls'

# Read the excel file
link_df = pd.read_excel(file_path)

link_df = link_df.rename(columns={'TICKER': 'oftic'})

# Merge Compustat and IBES on the matched GVKEYs and TICKERs
merged_ibes_df = ibes_data.merge(link_df ,on=['oftic'], how='left')

merged_ibes_df


,ticker,cusip,oftic,cname,statpers,measure,fiscalp,fpi,estflag,curcode,...,lowest,usfirm,fpedats,actual,actdats_act,acttims_act,anndats_act,anntims_act,curr_act,gvkey
0,0000,87482X10,TLMR,TALMER BANCORP,2014-04-17,EPS,ANN,1,P,USD,...,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD,NaN
1,0000,87482X10,TLMR,TALMER BANCORP,2014-05-15,EPS,ANN,1,P,USD,...,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD,NaN
2,0000,87482X10,TLMR,TALMER BANCORP,2014-06-19,EPS,ANN,1,P,USD,...,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD,NaN
3,0000,87482X10,TLMR,TALMER BANCORP,2014-07-17,EPS,ANN,1,P,USD,...,0.50,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD,NaN
4,0000,87482X10,TLMR,TALMER BANCORP,2014-08-14,EPS,ANN,1,P,USD,...,1.09,1.0,2014-12-31,1.21,2015-01-30,60887.0,2015-01-30,59400.0,USD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718807,ZYNX,98986M10,ZYXI,ZYNEX,2022-10-20,EPS,ANN,1,P,USD,...,0.41,1.0,2022-12-31,0.44,2023-03-13,29279.0,2023-03-13,28800.0,USD,NaN
718808,ZYNX,98986M10,ZYXI,ZYNEX,2022-11-17,EPS,ANN,1,P,USD,...,0.42,1.0,2022-12-31,0.44,2023-03-13,29279.0,2023-03-13,28800.0,USD,NaN
718809,ZYNX,98986M10,ZYXI,ZYNEX,2022-12-15,EPS,ANN,1,P,USD,...,0.42,1.0,2022-12-31,0.44,2023-03-13,29279.0,2023-03-13,28800.0,USD,NaN
718810,ZYNX,98986M10,ZYXI,ZYNEX,2023-01-19,EPS,ANN,1,P,USD,...,0.42,1.0,2022-12-31,0.44,2023-03-13,29279.0,2023-03-13,28800.0,USD,NaN


In [268]:
# Pick 10 stocks 
# Strategy to pick 10 stocks - select firms with highest Total Assets and Return on Assets in 2019

# Filter for fyear 2019
df_win_new_2019 = df_win_new[df_win_new['fyear'] == 2019]

# Sort by ROA and AT, descending
df_win_new_2019_sorted = df_win_new_2019.sort_values(by=['sale'], ascending=False)

# Remove rows with infinite values
df_win_new_2019_sorted = df_win_new_2019_sorted.replace([np.inf, -np.inf], np.nan).dropna()

# Get top 10 firms
tickers = ['MSFT', 'BP', 'AAPL', 'F', 'JNJ', 'WMT', 'INTC', 'CLSD', 'CPRX', 'CRSP']
mask = df_win_new_2019_sorted['tic'].isin(tickers)
top_10 = df_win_new_2019_sorted[mask]

# Print the results
print(top_10[['tic', 'conm', 'gvkey']])

        tic                          conm   gvkey
58872  CPRX  CATALYST PHARMACEUTICALS INC  175966
25895  CRSP        CRISPR THERAPEUTICS AG  028113
24740  CLSD      CLEARSIDE BIOMEDICAL INC  026798
10250  MSFT                MICROSOFT CORP  012141
9505    WMT                   WALMART INC  011259
4625   INTC                    INTEL CORP  006008
4882    JNJ             JOHNSON & JOHNSON  006266
608    AAPL                     APPLE INC  001690
3590      F                 FORD MOTOR CO  004839
1284     BP                        BP PLC  002410


In [269]:
# Using E+1 modified model in Q2 to estimate/predict Earnings for 10 stocks
# Make predictions using the model for 2020 
top_10 = top_10.set_index('gvkey')
y_pred_2020_top_10 = top_10.apply(lambda row: params_hvz_model_new_t1['intercept'] + \
    params_hvz_model_new_t1['at']*row['at'] + params_hvz_model_new_t1['dvc']*row['dvc'] + \
    params_hvz_model_new_t1['E_t']*row['E_t'] + params_hvz_model_new_t1['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_new_t1['ACT']*row['ACT'] + params_hvz_model_new_t1['div_payout_ratio']*row['div_payout_ratio'] + \
    params_hvz_model_new_t1['total_debt']*row['total_debt'] + params_hvz_model_new_t1['deq_ratio']*row['deq_ratio'] + \
    params_hvz_model_new_t1['roa']*row['roa'] + params_hvz_model_new_t1['oancf']*row['oancf'] + \
    params_hvz_model_new_t1['rdip']*row['rdip'] + params_hvz_model_new_t1['invch']*row['invch'] + \
    params_hvz_model_new_t1['sale']*row['sale'], axis=1)

# convert the Series to a DataFrame
y_pred_2020_top_10 =y_pred_2020_top_10.to_frame()
# Rename the predicted column
y_pred_2020_top_10 = y_pred_2020_top_10.rename(columns={0: 'predicted_earnings'})

merged_df_top_10 = top_10.merge(y_pred_2020_top_10, on='gvkey')

merged_df_top_10 = merged_df_top_10[['tic', 'conm', 'predicted_earnings']]
merged_df_top_10

,tic,conm,predicted_earnings
gvkey,,,
175966,CPRX,CATALYST PHARMACEUTICALS INC,36.033368
028113,CRSP,CRISPR THERAPEUTICS AG,52.231213
026798,CLSD,CLEARSIDE BIOMEDICAL INC,-20.884361
012141,MSFT,MICROSOFT CORP,35362.660582
011259,WMT,WALMART INC,14513.488450
006008,INTC,INTEL CORP,18867.185373
006266,JNJ,JOHNSON & JOHNSON,16168.004649
001690,AAPL,APPLE INC,50308.208812
004839,F,FORD MOTOR CO,4863.262953


In [270]:
# Filter the dataframe to include only the data for the year-end 2020
merged_ibes_df['statpers'] = merged_ibes_df['statpers'].astype(str)
df_2020 = merged_ibes_df[merged_ibes_df['statpers'].str.startswith('2020-06')]
df_2020

,ticker,cusip,oftic,cname,statpers,measure,fiscalp,fpi,estflag,curcode,...,lowest,usfirm,fpedats,actual,actdats_act,acttims_act,anndats_act,anntims_act,curr_act,gvkey
272,000V,28249U10,EIGR,EIGER,2020-06-18,EPS,ANN,1,P,USD,...,-2.87,1.0,2020-06-18,-2.3100,2021-03-09,60684.0,2021-03-09,58740.0,USD,NaN
379,000Y,90400D10,RARE,ULTRAGENYX,2020-06-18,EPS,ANN,1,P,USD,...,-7.46,1.0,2020-06-18,-3.0700,2021-02-11,59544.0,2021-02-11,57900.0,USD,30001.0
484,000Z,09072V40,BIOC,BIOCEPT,2020-06-18,EPS,ANN,1,P,USD,...,-2.70,1.0,2020-06-18,-1.5000,2021-03-29,58592.0,2021-03-29,57900.0,USD,2224.0
679,001A,81776310,SESN,SESEN BIO,2020-06-18,EPS,ANN,1,P,USD,...,-0.60,1.0,2020-06-18,-3.8000,2021-03-15,27795.0,2021-03-15,25200.0,USD,NaN
834,001J,49926D10,KN,KNOWLES,2020-06-18,EPS,ANN,1,P,USD,...,0.49,1.0,2020-06-18,0.6400,2021-02-04,62016.0,2021-02-04,57900.0,USD,6387.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718347,ZUMZ,98981710,ZUMZ,ZUMIEZ,2020-06-18,EPS,ANN,1,P,USD,...,0.80,1.0,2020-06-18,3.0000,2021-03-11,63307.0,2021-03-11,58200.0,USD,162988.0
718393,ZUO,98983V10,ZUO,ZUORA,2020-06-18,EPS,ANN,1,P,USD,...,-0.26,1.0,2020-06-18,-0.0900,2021-03-11,63497.0,2021-03-11,58020.0,USD,33232.0
718601,ZY,87254010,TJX,TJX,2020-06-18,EPS,ANN,1,P,USD,...,-0.47,1.0,2020-06-18,0.3100,2021-02-24,28801.0,2021-02-24,27480.0,USD,13504.0
718679,ZYNE,98986X10,ZYNE,ZYNERBA PHARMS,2020-06-18,EPS,ANN,1,P,USD,...,-1.99,1.0,2020-06-18,-1.9000,2021-03-10,24946.0,2021-03-10,24300.0,USD,25128.0


In [271]:
# Filter df_2020 to include only the tickers that are in the top_10 dataframe
df_2020_top_10 = df_2020[df_2020['ticker'].isin(merged_df_top_10['tic'])]
df_2020_top_10 = df_2020_top_10[['ticker','cname','medest']]
df_2020_top_10 = df_2020_top_10.rename(columns={'ticker': 'tic'})
df_2020_top_10

,tic,cname,medest
40118,AAPL,APPLE,3.09
127036,BP,BP,0.04
167715,CLSD,CLEARSIDE,-0.40
182531,CPRX,CATALYST PHARMS,0.39
187399,CRSP,CRISPR THERAPEUT,-4.66
255928,F,FORD MOTOR,-1.20
352168,INTC,INTEL,4.79
366173,JNJ,JOHNSON & JOHNSO,7.72
438326,MSFT,MICROSOFT,5.69
698170,WMT,WALMART,5.00


In [272]:
# Filter the dataframe on fyear 2020 and the list of tickers
df_filtered = delta_df[(delta_df['fyear']==2020) & (delta_df['tic'].isin(tickers))]

# Select only the required columns
df_filtered = df_filtered.loc[:, ['tic', 'csho', 'prcc_f']]
df_filtered

,tic,csho,prcc_f
1053,AAPL,16976.763,115.81
2099,BP,3377.338,20.52
5640,F,3978.695,8.79
7343,INTC,4062.000,49.82
7712,JNJ,2632.512,157.38
15083,WMT,2821.000,140.49
16396,MSFT,7571.000,203.51
50397,CLSD,51.861,2.74
53611,CRSP,73.915,153.11
126262,CPRX,103.782,3.34


In [274]:
# Standardizing predicted earnings and analyst forecast to compare EPS of the 10 firms selected
# merge the dataframes on the 'tic' column
merged_df = pd.merge(df_filtered, df_2020_top_10, on='tic', how='outer')
merged_df = pd.merge(merged_df, merged_df_top_10, on='tic', how='outer')
merged_df.drop('cname', axis=1, inplace=True)
merged_df

,tic,csho,prcc_f,medest,conm,predicted_earnings
0,AAPL,16976.763,115.81,3.09,APPLE INC,50308.208812
1,BP,3377.338,20.52,0.04,BP PLC,9034.374379
2,F,3978.695,8.79,-1.20,FORD MOTOR CO,4863.262953
3,INTC,4062.000,49.82,4.79,INTEL CORP,18867.185373
4,JNJ,2632.512,157.38,7.72,JOHNSON & JOHNSON,16168.004649
5,WMT,2821.000,140.49,5.00,WALMART INC,14513.488450
6,MSFT,7571.000,203.51,5.69,MICROSOFT CORP,35362.660582
7,CLSD,51.861,2.74,-0.40,CLEARSIDE BIOMEDICAL INC,-20.884361
8,CRSP,73.915,153.11,-4.66,CRISPR THERAPEUTICS AG,52.231213
9,CPRX,103.782,3.34,0.39,CATALYST PHARMACEUTICALS INC,36.033368


In [275]:
# Dividing predicted_earnings by (csho*prcc_f) and medest by prcc_f
merged_df['scaled_predicted_EPS'] = merged_df['predicted_earnings']/(merged_df['csho']*merged_df['prcc_f'])
merged_df['scaled_analyst_forecast'] = merged_df['medest']/merged_df['prcc_f']
merged_df['diff'] = merged_df['scaled_predicted_EPS'] - merged_df['scaled_analyst_forecast']
merged_df

,tic,csho,prcc_f,medest,conm,predicted_earnings,scaled_predicted_EPS,scaled_analyst_forecast,diff
0,AAPL,16976.763,115.81,3.09,APPLE INC,50308.208812,0.025588,0.026682,-0.001094
1,BP,3377.338,20.52,0.04,BP PLC,9034.374379,0.130361,0.001949,0.128411
2,F,3978.695,8.79,-1.20,FORD MOTOR CO,4863.262953,0.139059,-0.136519,0.275577
3,INTC,4062.000,49.82,4.79,INTEL CORP,18867.185373,0.093232,0.096146,-0.002914
4,JNJ,2632.512,157.38,7.72,JOHNSON & JOHNSON,16168.004649,0.039024,0.049053,-0.010029
5,WMT,2821.000,140.49,5.00,WALMART INC,14513.488450,0.036620,0.035590,0.001031
6,MSFT,7571.000,203.51,5.69,MICROSOFT CORP,35362.660582,0.022951,0.027959,-0.005008
7,CLSD,51.861,2.74,-0.40,CLEARSIDE BIOMEDICAL INC,-20.884361,-0.146970,-0.145985,-0.000985
8,CRSP,73.915,153.11,-4.66,CRISPR THERAPEUTICS AG,52.231213,0.004615,-0.030436,0.035051
9,CPRX,103.782,3.34,0.39,CATALYST PHARMACEUTICALS INC,36.033368,0.103953,0.116766,-0.012814


In [276]:
mae = mean_absolute_error(merged_df['scaled_predicted_EPS'], merged_df['scaled_analyst_forecast'])
mse = mean_squared_error(merged_df['scaled_predicted_EPS'], merged_df['scaled_analyst_forecast'])

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

MAE: 0.0473
MSE: 0.0094


#### Applying coefficients estimated above to 2022 data to predict the top 10 firms that are most likely to exhibit the largest earnings growth in 2023. 

In [291]:
# Applying coefficients of E_t+1 model estimated in Q2 to predict earnings for 2023. 
# Note: The modified model in Q2 for E_t+1 is trained till 2018 data and is used to predict earnings for 2020.
#       Here as the question asks, we have used this model to predict 2023 earnings using data from 2022.

# Select the same columns as used in training the model
cols_to_check_for_inf = ['at','dvc','E_t','dummy_Neg_E_t','ACT','div_payout_ratio',
                                                 'total_debt','deq_ratio','roa','oancf','rdip','invch','sale']
# Remove rows with infinite values
hvz_final = test_data_new_2022[cols_to_check_for_inf+['gvkey']].replace([np.inf, -np.inf], np.nan)
hvz_final = hvz_final.dropna(subset=cols_to_check_for_inf)
hvz_final = hvz_final.set_index('gvkey')

# Make predictions using the model for 2020 data
y_pred_2023 = hvz_final.apply(lambda row: params_hvz_model_new_t1['intercept'] + \
    params_hvz_model_new_t1['at']*row['at'] + params_hvz_model_new_t1['dvc']*row['dvc'] + \
    params_hvz_model_new_t1['E_t']*row['E_t'] + params_hvz_model_new_t1['dummy_Neg_E_t']*row['dummy_Neg_E_t'] + \
    params_hvz_model_new_t1['ACT']*row['ACT'] + params_hvz_model_new_t1['div_payout_ratio']*row['div_payout_ratio'] + \
    params_hvz_model_new_t1['total_debt']*row['total_debt'] + params_hvz_model_new_t1['deq_ratio']*row['deq_ratio'] + \
    params_hvz_model_new_t1['roa']*row['roa'] + params_hvz_model_new_t1['oancf']*row['oancf'] + \
    params_hvz_model_new_t1['rdip']*row['rdip'] + params_hvz_model_new_t1['invch']*row['invch'] + \
    params_hvz_model_new_t1['sale']*row['sale'], axis=1)

# Reset the index of the y_pred_2020 DataFrame
y_pred_2023= y_pred_2023.reset_index()
# Rename the predicted column
y_pred_2023 = y_pred_2023.rename(columns={0: 'predicted_earnings_2023'})
y_pred_2023

,gvkey,predicted_earnings_2023
0,001050,35.536727
1,001075,636.824590
2,001078,7038.340957
3,001096,234.369765
4,001104,15.313474
...,...,...
3603,345699,-135.493754
3604,345764,-5.157191
3605,345920,-67.316008
3606,345980,-289.615473


In [294]:
# Calculating Earnings Growth using year 2022 earnings and predicted earnings for 2023 
growth_df = pd.merge(y_pred_2023, test_data_new_2022[['gvkey','E_t']], on='gvkey', how='left')


growth_df['earnings_growth'] = (growth_df['predicted_earnings_2023'] - growth_df['E_t']) / growth_df['E_t']
top_10_earnings_growth = growth_df[['gvkey', 'earnings_growth']].sort_values('earnings_growth', ascending=False).head(10)
top_10_earnings_growth

,gvkey,earnings_growth
683,016564,8877.491611
33,001718,3031.387159
3165,175548,927.862226
704,017349,337.163856
1097,024368,125.695625
961,021563,108.650485
2345,060801,105.988442
2853,143788,102.290555
37,001783,97.651521
493,011372,80.211035


In [296]:
# Get company names for these gvkeys
top_10_firms_final = pd.merge(top_10_earnings_growth, test_data_new_2022[['gvkey', 'conm']], on='gvkey', how='left')
top_10_firms_final

,gvkey,earnings_growth,conm
0,016564,8877.491611,WALL STREET MEDIA CO
1,001718,3031.387159,ADVANCED OXYGEN TECHNOLOGY
2,175548,927.862226,NEXGENRX INC
3,017349,337.163856,SKKYNET CLOUD SYSTEMS INC
4,024368,125.695625,RETAIL HOLDINGS NV
5,021563,108.650485,ELECTRONIC SYSTEM TECH INC
6,060801,105.988442,SOCKET MOBILE INC
7,143788,102.290555,REFLECT SCIENTIFIC INC
8,001783,97.651521,ARTS WAY MFG INC
9,011372,80.211035,NEW CONCEPT ENERGY INC
